In [1]:
from py_sc_fermi.defect_charge_state import DefectChargeState, FrozenDefectChargeState
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files, read_input_data
from scipy.constants import physical_constants
from copy import deepcopy
import pandas as pd
import numpy as np
import auto_functions as af
import matplotlib.pyplot as plt
import yaml

### import physical constants ###  

kb_e = physical_constants['Boltzmann constant in eV/K'][0]

### import defect data ###

with open('automator_config.yaml', 'r') as stream:   # get defect information from .yaml config
    try:                                           
        defect_params = yaml.safe_load(stream)       # return as a dict called defect_params
    except yaml.YAMLError as exc:
        print(exc)
        
### import DFT data ###

defects = af.import_calculations_from_file('defects.yaml')         # read in raw vasp data as vasppy calculation objects.
elements = af.import_calculations_from_file('elements.yaml')       # and give them intuitive names
llzo = af.import_calculations_from_file('interest.yaml')           #
ex_grid = (pd.read_csv('ex_grid.dat', skiprows=10, sep='\s+'))     # read in chemical potential grids as outputted by cplap 
grid = (pd.read_csv('grid.dat', skiprows=10, sep='\s+'))           # with both a larger grid and a smaller grid


def make_all_defects(config_dict,chem_pots):
    """
    Take defect data and return py_sc_fermi friendly defect objects.
    """
    defs = []
    for k,v in config_dict['defects'].items():
        label = [i for i in config_dict['defects'][k][1:]]
        to_calcs = [defects[i] for i in label]
        out = af.make_defect(to_calcs, elements, llzo['LLZO'], delta_mu=chem_pots, corr=config_dict['iccs'], sites=1)
        out._nsites = config_dict['defects'][out.name][0]
        defs.append(out)
    return defs

unitcell_filename = 'unitcell.dat'            # Read structure
totdos_filename = 'totdos.dat'                # read totdos.dat
input_fermi_filename = 'input_fermi_cont.dat' # read default calculation params

charge_states = {'v_O':[0,1,2], # this is just a lazy labelling excercise. 
 'O_i':[0,-1],
 'v_La':[-3],
 'v_Zr':[-4],
 'Zr_i':[0,1,2,3,4],
 'Zr_Li_tet':[0,1,2,3],
 'Zr_Li':[0,1,2,3],
 'Zr_La':[0,1],
 'La_Zr':[0,-1],
 'La_Li':[0,1,2],
 'Li_La':[0,-1,-2],
 'Li_Zr':[0,-1,-2,-3]}

compilied_defects = make_all_defects(defect_params,grid.iloc[45])         # make defects for some point in the stability region
reference_defects = deepcopy(compilied_defects)                           # make a copy of the original defects 
inputs = inputs_from_files(unitcell_filename=unitcell_filename,           # volume from file
                           input_fermi_filename=input_fermi_filename,     # inputs from file (these will be overwritten)
                           totdos_filename=totdos_filename, verbose=False)               # dos from file

In [11]:
defect_system_reference = DefectSystem(defect_species=reference_defects,  # defects = copy of defects
                                       volume=inputs['volume'],           # volume from inputs
                                       dos=inputs['dos'],                 # dos from inputs
                                       temperature=700,                   # high T run
                                       spin_pol=2)                        # spin polarised run

reference_concentrations = defect_system_reference.to_dict_per_site()                   # get defect concentrations per site as a dictionary
sum_constraint = reference_concentrations['v_Li'] + reference_concentrations['Li_i']    # take the constraint sum as the difference between lithium vacancy and interstitials
constrained_fermi_energy = defect_system_reference.get_constrained_sc_fermi(constraint={'v_Li': +1, 'Li_i': -1}, total=sum_constraint) # get constrained SC-Fermi
unconstrained_fermi_energy = defect_system_reference.to_dict_per_site()['Fermi Energy']

print(f'Unconstrained fermi energy at {defect_system_reference.temperature} K is {unconstrained_fermi_energy} eV')
print(f'Constrained fermi energy at {defect_system_reference.temperature} K is {constrained_fermi_energy} eV')

constrained_concentrations = {str(defect.name):defect.charge_state_concentrations(constrained_fermi_energy,700) for defect in defect_system_reference.defect_species}  # Concentrations based on constrained Fermi Energy
fixed_defects = [DefectSpecies(defect,1,[FrozenDefectChargeState(charge,constrained_concentrations[defect][charge]) for charge in charges]) for defect,charges in charge_states.items()]  # Create fixed conc defects based on constrained Fermi energy

defect_system_constrained = DefectSystem(defect_species=fixed_defects,volume=inputs['volume'],dos=inputs['dos'],temperature=700,spin_pol=2)
frozen_constraint_fermi_energy = defect_system_constrained.to_dict_per_site()['Fermi Energy']

print(f'Defects frozen at constratined concentration fermi energy at {defect_system_constrained.temperature} K is {frozen_constraint_fermi_energy} eV')

Unconstrained fermi energy at 700 K is 3.316466389302633 eV
Constrained fermi energy at 700 K is 3.316298023189411 eV
Defects frozen at constratined concentration fermi energy at 700 K is 1.469937528610231 eV
